# This notebook aims at evaluating the size of networks:

In [1]:
import numpy as np
import torch
import torch.nn as nn
import configparser as ConfigParser

## Local files imports:
from Models import MLP, flip
from Models import SincNet as CNN
from Models import SincNet2D as CNN2D
from read_conf_files import str_to_bool
from utils import Dataset, LoadPrevModel
from training_and_acc_fun import accuracy


In [2]:
class Options(object):

    def __init__(self, cfg):
        '''Defines the cfg file'''
        self.cfg = cfg

In [3]:
def read_conf(config_file_path):
    # Removed the possibility of executing with --cfg = path
    # Instead we feed it directly

    # Initializing dummy class with cfg folder path
    options = Options(config_file_path)

    # Reading the config file with config parser
    Config = ConfigParser.ConfigParser()
    Config.read(options.cfg)

    #[data]
    options.tr_lst=Config.get('data', 'tr_lst')
    options.te_lst=Config.get('data', 'te_lst')
    options.lab_dict=Config.get('data', 'lab_dict')
    options.data_folder=Config.get('data', 'data_folder')
    options.output_folder=Config.get('data', 'output_folder')
    options.pt_file=Config.get('data', 'pt_file')

    #[windowing]
    options.fs=Config.get('windowing', 'fs')
    options.cw_len=Config.get('windowing', 'cw_len')
    options.cw_shift=Config.get('windowing', 'cw_shift')

    if('cnn2D' in Config.sections()):
        #[cnn2D]
        options.is_conv2D = True
        options.cnn_N_filt=Config.get('cnn2D', 'cnn_N_filt')
        options.cnn_len_filt_W=Config.get('cnn2D', 'cnn_len_filt_W')
        options.cnn_len_filt_H=Config.get('cnn2D', 'cnn_len_filt_H')
        options.cnn_energy_L=Config.get('cnn2D', 'cnn_energy_L')
        options.cnn_energy_stride=Config.get('cnn2D', 'cnn_energy_stride')
        options.cnn_max_pool_len_W=Config.get('cnn2D', 'cnn_max_pool_len_W')
        options.cnn_max_pool_len_H=Config.get('cnn2D', 'cnn_max_pool_len_H')
        options.cnn_use_laynorm_inp=Config.get('cnn2D', 'cnn_use_laynorm_inp')
        options.cnn_use_batchnorm_inp=Config.get('cnn2D', 'cnn_use_batchnorm_inp')
        options.cnn_use_laynorm=Config.get('cnn2D', 'cnn_use_laynorm')
        options.cnn_use_batchnorm=Config.get('cnn2D', 'cnn_use_batchnorm')
        options.cnn_act=Config.get('cnn2D', 'cnn_act')
        options.cnn_drop=Config.get('cnn2D', 'cnn_drop')
    else:
        #[cnn]
        options.is_conv2D = False
        options.cnn_N_filt=Config.get('cnn', 'cnn_N_filt')
        options.cnn_len_filt=Config.get('cnn', 'cnn_len_filt')
        options.cnn_max_pool_len=Config.get('cnn', 'cnn_max_pool_len')
        options.cnn_use_laynorm_inp=Config.get('cnn', 'cnn_use_laynorm_inp')
        options.cnn_use_batchnorm_inp=Config.get('cnn', 'cnn_use_batchnorm_inp')
        options.cnn_use_laynorm=Config.get('cnn', 'cnn_use_laynorm')
        options.cnn_use_batchnorm=Config.get('cnn', 'cnn_use_batchnorm')
        options.cnn_act=Config.get('cnn', 'cnn_act')
        options.cnn_drop=Config.get('cnn', 'cnn_drop')


    #[dnn]
    options.fc_lay=Config.get('dnn', 'fc_lay')
    options.fc_drop=Config.get('dnn', 'fc_drop')
    options.fc_use_laynorm_inp=Config.get('dnn', 'fc_use_laynorm_inp')
    options.fc_use_batchnorm_inp=Config.get('dnn', 'fc_use_batchnorm_inp')
    options.fc_use_batchnorm=Config.get('dnn', 'fc_use_batchnorm')
    options.fc_use_laynorm=Config.get('dnn', 'fc_use_laynorm')
    options.fc_act=Config.get('dnn', 'fc_act')

    #[class]
    options.class_lay=Config.get('class', 'class_lay')
    options.class_drop=Config.get('class', 'class_drop')
    options.class_use_laynorm_inp=Config.get('class', 'class_use_laynorm_inp')
    options.class_use_batchnorm_inp=Config.get('class', 'class_use_batchnorm_inp')
    options.class_use_batchnorm=Config.get('class', 'class_use_batchnorm')
    options.class_use_laynorm=Config.get('class', 'class_use_laynorm')
    options.class_act=Config.get('class', 'class_act')

    #[optimization]
    if('optimization' in Config.sections()):
        options.lr=Config.get('optimization', 'lr')

        ## use_scheduler:
        if 'use_scheduler' in Config['optimization']:
            options.use_scheduler=Config.get('optimization', 'use_scheduler')
        else:
            print("You did not specify the value of `use_scheduler`, it is set to False.")
            options.use_scheduler='False'

        ## scheduler_patience:
        if 'scheduler_patience' in Config['optimization']:
            options.scheduler_patience=Config.get('optimization', 'scheduler_patience')
        else:
            options.scheduler_patience=2
            print("You did not specify the value of `scheduler_patience`, it is set to {}.".format(options.scheduler_patience))

        ## scheduler_factor:
        if 'scheduler_factor' in Config['optimization']:
            options.scheduler_factor=Config.get('optimization', 'scheduler_factor')
        else:
            options.scheduler_factor=0.5
            print("You did not specify the value of `scheduler_factor`, it is set to {}.".format(options.scheduler_factor))


        options.batch_size=Config.get('optimization', 'batch_size')

        ## Batch_dev:
        if 'Batch_dev' in Config['optimization']:
            options.Batch_dev=Config.get('optimization', 'Batch_dev')
        else:
            options.Batch_dev=32
            print("You did not specify the value of `Batch_dev`, it is set to {}.".format(options.Batch_dev))

        ## patience:
        if 'patience' in Config['optimization']:
            options.patience=Config.get('optimization', 'patience')
        else:
            print("You did not specify the value of `patience`, it is set to 7.")
            options.patience=7

        options.N_epochs=Config.get('optimization', 'N_epochs')
        options.N_batches=Config.get('optimization', 'N_batches')
        options.N_eval_epoch=Config.get('optimization', 'N_eval_epoch')
        
        ## train_acc_period:
        if 'train_acc_period' in Config['optimization']:
                options.train_acc_period=Config.get('optimization', 'train_acc_period')
        else:
            print("You did not specify the value of `train_acc_period`, it is set to 5.")
            options.train_acc_period=5
        
        ## fact_amp:        
        if 'fact_amp' in Config['optimization']:
                options.fact_amp=Config.get('optimization', 'fact_amp')
        else:
            options.fact_amp=0.2
            print("You did not specify the value of `fact_amp`, it is set to {}.".format(options.fact_amp))
        
        ## use_mixup:
        if 'use_mixup' in Config['optimization']:
            options.use_mixup=Config.get('optimization', 'use_mixup')
        else:
            print("You did not specify the value of `use_mixup`, it is set to False.")
            options.use_mixup='False'
        
        ## mixup_batch_prop:        
        if 'mixup_batch_prop' in Config['optimization']:
            options.mixup_batch_prop=Config.get('optimization', 'mixup_batch_prop')
        else:
            options.mixup_batch_prop=float(1.0) if options.use_mixup=='True' else float(0.0)
            print("You did not specify the value of `mixup_batch_prop`, it is set to {}%.".format(options.mixup_batch_prop*100))
        
        ## beta_coef:
        if 'beta_coef' in Config['optimization']:
            options.beta_coef=Config.get('optimization', 'beta_coef')
        else:
            print("You did not specify the value of `beta_coef`, it is set to 0.4.")
            options.beta_coef=0.4
        
        ## same_classes:        
        if 'same_classes' in Config['optimization']:
            options.same_classes=Config.get('optimization', 'same_classes')
        else:
            options.same_classes='False'
            print("You did not specify the value of `same_classes`, it is set to {}.".format(options.same_classes))
            if("True" in options.use_mixup):
                print("Warning: you are using mixup but you did not mention which type in config file. \n"+
                    "By default it will be set to False. You are advised to add a same_class attribute to your cfg file and set it to True or False.")    

            
        options.seed=Config.get('optimization', 'seed')
    else:
        print("Error, you did not specify optimization parameters in your cfg. Consequently, the code won't run...")

    return options


In [4]:
# Config path location
config_file_path = "cfg/SincNet2D/SincNet2D_CNNLay4_Rand0PreEnergyWindow3000_Scheduler_PReLu_Drop30.cfg"

# Reading cfg file and storing its parameters into options :
options=read_conf(config_file_path)

print(options.output_folder)
print(options.data_folder)

You did not specify the value of `fact_amp`, it is set to 0.2.
You did not specify the value of `mixup_batch_prop`, it is set to 0.0%.
exp/SincNet2D_DCASE/CNNlay4_Rand0PreEnergy4000ms_Scheduler0.2_Window3000ms_PReLu_Drop30
Data/Audio_Tensors/Train/Preprocessed_withEnergy_AudioTensors_Window4000ms_Random0Padding/


In [5]:
## <!>---------------------------- Reading the config file ----------------------------<!> ##
# Reading cfg file
options=read_conf(config_file_path)

## Architecture of the file:
#[data]
pt_file=options.pt_file
output_folder=options.output_folder

#[windowing]
fs=int(options.fs)
cw_len=int(options.cw_len)
cw_shift=int(options.cw_shift)

is_conv2D = options.is_conv2D
conv_type = '2D' if is_conv2D else '1D'
print("The file contains the config of a {} convolutional network.".format(conv_type))
if is_conv2D:
    #[cnn2D]
    cnn_N_filt=list(map(int, options.cnn_N_filt.split(',')))
    cnn_len_filt_W=list(map(int, options.cnn_len_filt_W.split(',')))
    cnn_len_filt_H=list(map(int, options.cnn_len_filt_H.split(',')))
    cnn_energy_L=int(options.cnn_energy_L)
    cnn_energy_stride=int(options.cnn_energy_stride)
    cnn_max_pool_len_W=list(map(int, options.cnn_max_pool_len_W.split(',')))
    cnn_max_pool_len_H=list(map(int, options.cnn_max_pool_len_H.split(',')))
else:
    #[cnn]
    cnn_N_filt=list(map(int, options.cnn_N_filt.split(',')))
    cnn_len_filt=list(map(int, options.cnn_len_filt.split(',')))
    cnn_max_pool_len=list(map(int, options.cnn_max_pool_len.split(',')))

cnn_use_laynorm_inp=str_to_bool(options.cnn_use_laynorm_inp)
cnn_use_batchnorm_inp=str_to_bool(options.cnn_use_batchnorm_inp)
cnn_use_laynorm=list(map(str_to_bool, options.cnn_use_laynorm.split(',')))
cnn_use_batchnorm=list(map(str_to_bool, options.cnn_use_batchnorm.split(',')))
cnn_act=list(map(str, options.cnn_act.split(',')))
cnn_drop=list(map(float, options.cnn_drop.split(',')))

#[dnn]
fc_lay=list(map(int, options.fc_lay.split(',')))
fc_drop=list(map(float, options.fc_drop.split(',')))
fc_use_laynorm_inp=str_to_bool(options.fc_use_laynorm_inp)
fc_use_batchnorm_inp=str_to_bool(options.fc_use_batchnorm_inp)
fc_use_batchnorm=list(map(str_to_bool, options.fc_use_batchnorm.split(',')))
fc_use_laynorm=list(map(str_to_bool, options.fc_use_laynorm.split(',')))
fc_act=list(map(str, options.fc_act.split(',')))

#[class]
class_lay=list(map(int, options.class_lay.split(',')))
class_drop=list(map(float, options.class_drop.split(',')))
class_use_laynorm_inp=str_to_bool(options.class_use_laynorm_inp)
class_use_batchnorm_inp=str_to_bool(options.class_use_batchnorm_inp)
class_use_batchnorm=list(map(str_to_bool, options.class_use_batchnorm.split(',')))
class_use_laynorm=list(map(str_to_bool, options.class_use_laynorm.split(',')))
class_act=list(map(str, options.class_act.split(',')))

#[optimization]
Batch_dev=int(options.Batch_dev)


# Converting context and shift in samples
wlen=int(fs*cw_len/1000.00)
wshift=int(fs*cw_shift/1000.00)

You did not specify the value of `fact_amp`, it is set to 0.2.
You did not specify the value of `mixup_batch_prop`, it is set to 0.0%.
The file contains the config of a 2D convolutional network.


In [6]:
## Setting cuda Device
print("Selecting Cuda device... \t\t", end="")
Desired_cuda_device_number = int(1)

if torch.cuda.is_available(): # we'll use cuda
    device = "cuda:"+str(Desired_cuda_device_number)
    torch.cuda.set_device(device)
    if(torch.cuda.current_device() == Desired_cuda_device_number and torch.cuda.is_available()):
        print("Cuda device {} was selected successfully!".format(Desired_cuda_device_number))
    else:
        print("Cuda was not selected successfully...")
else:
    print("Cuda device(s) is(are) not available.")

Selecting Cuda device... 		Cuda device 1 was selected successfully!


In [7]:
# loss functions
cost = nn.NLLLoss()

In [8]:
## <!>------------------- Initializing the Networks with .cfg options -------------------<!> ##

print("Initializing the Networks... \t\t", end="")
# Feature extractor CNN
if is_conv2D:
    CNN_arch = {'input_dim': wlen,
            'fs': fs,
            'cnn_N_filt': cnn_N_filt,
            'cnn_len_filt_W': cnn_len_filt_W,
            'cnn_len_filt_H': cnn_len_filt_H,
            'cnn_energy_L': cnn_energy_L,
            'cnn_energy_stride': cnn_energy_stride,
            'cnn_max_pool_len_W': cnn_max_pool_len_W,
            'cnn_max_pool_len_H': cnn_max_pool_len_H,
            'cnn_use_laynorm_inp': cnn_use_laynorm_inp,
            'cnn_use_batchnorm_inp': cnn_use_batchnorm_inp,
            'cnn_use_laynorm':cnn_use_laynorm,
            'cnn_use_batchnorm':cnn_use_batchnorm,
            'cnn_act': cnn_act,
            'cnn_drop':cnn_drop,          
            }
else:
    CNN_arch = {'input_dim': wlen,
            'fs': fs,
            'cnn_N_filt': cnn_N_filt,
            'cnn_len_filt': cnn_len_filt,
            'cnn_max_pool_len':cnn_max_pool_len,
            'cnn_use_laynorm_inp': cnn_use_laynorm_inp,
            'cnn_use_batchnorm_inp': cnn_use_batchnorm_inp,
            'cnn_use_laynorm':cnn_use_laynorm,
            'cnn_use_batchnorm':cnn_use_batchnorm,
            'cnn_act': cnn_act,
            'cnn_drop':cnn_drop,          
            }

## Initializes SincNet:
CNN_net=CNN2D(CNN_arch) if is_conv2D else CNN(CNN_arch)
CNN_net.cuda()

#fc_lay = [2048]*3

## First DNN, follows the config from the section [dnn] in .cfg file
DNN1_arch = {'input_dim': CNN_net.out_dim,
          'fc_lay': fc_lay,
          'fc_drop': fc_drop, 
          'fc_use_batchnorm': fc_use_batchnorm,
          'fc_use_laynorm': fc_use_laynorm,
          'fc_use_laynorm_inp': fc_use_laynorm_inp,
          'fc_use_batchnorm_inp':fc_use_batchnorm_inp,
          'fc_act': fc_act,
          }

DNN1_net=MLP(DNN1_arch)
DNN1_net.cuda()

## Last trainable layer, has softmax as activation function see section [class] in .cfg
DNN2_arch = {'input_dim':fc_lay[-1] ,
          'fc_lay': class_lay,
          'fc_drop': class_drop, 
          'fc_use_batchnorm': class_use_batchnorm,
          'fc_use_laynorm': class_use_laynorm,
          'fc_use_laynorm_inp': class_use_laynorm_inp,
          'fc_use_batchnorm_inp':class_use_batchnorm_inp,
          'fc_act': class_act,
          }


DNN2_net=MLP(DNN2_arch)
DNN2_net.cuda()


print("Initialization done!")

Initializing the Networks... 		Initialization done!


In [9]:
class MainNet(nn.Module):

    def __init__(self, CNN_net, DNN1_net, DNN2_net):
        super(MainNet, self).__init__()
        self.CNN_net  = CNN_net
        self.DNN1_net = DNN1_net
        self.DNN2_net = DNN2_net

    def forward(self, x):
        x = self.DNN2_net(self.DNN1_net(self.CNN_net(x)))
        return x

In [10]:
Main_net = MainNet(CNN_net, DNN1_net, DNN2_net)
Main_net.cuda()

MainNet(
  (CNN_net): SincNet2D(
    (conv): ModuleList(
      (0): SincConv_fast()
      (1): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1))
      (2): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1))
      (3): Conv2d(20, 20, kernel_size=(3, 3), stride=(1, 1))
    )
    (bn): ModuleList(
      (0): BatchNorm1d(80, eps=1e-05, momentum=0.05, affine=True, track_running_stats=True)
      (1): BatchNorm2d(10, eps=1e-05, momentum=0.05, affine=True, track_running_stats=True)
      (2): BatchNorm2d(20, eps=1e-05, momentum=0.05, affine=True, track_running_stats=True)
      (3): BatchNorm2d(20, eps=1e-05, momentum=0.05, affine=True, track_running_stats=True)
    )
    (ln): ModuleList(
      (0): LayerNorm(torch.Size([80, 31916]), eps=1e-05, elementwise_affine=True)
      (1): LayerNorm(torch.Size([10, 39, 353]), eps=1e-05, elementwise_affine=True)
      (2): LayerNorm(torch.Size([20, 18, 117]), eps=1e-05, elementwise_affine=True)
      (3): LayerNorm(torch.Size([20, 16, 57]), eps=1e-05, el

In [11]:
CNN_arch

{'input_dim': 96000,
 'fs': 32000,
 'cnn_N_filt': [80, 10, 20, 20],
 'cnn_len_filt_W': [251, 3, 3, 3],
 'cnn_len_filt_H': [0, 3, 3, 3],
 'cnn_energy_L': 60,
 'cnn_energy_stride': 30,
 'cnn_max_pool_len_W': [3, 3, 3, 2],
 'cnn_max_pool_len_H': [1, 2, 2, 1],
 'cnn_use_laynorm_inp': True,
 'cnn_use_batchnorm_inp': False,
 'cnn_use_laynorm': [True, True, True, True],
 'cnn_use_batchnorm': [False, False, False, False],
 'cnn_act': ['leaky_relu', 'leaky_relu', 'leaky_relu', 'leaky_relu'],
 'cnn_drop': [0.0, 0.0, 0.0, 0.0]}

In [12]:
DNN1_arch

{'input_dim': 18240,
 'fc_lay': [1024, 1024, 512],
 'fc_drop': [0.3, 0.3, 0.3, 0.3],
 'fc_use_batchnorm': [True, True, True],
 'fc_use_laynorm': [False, False, False],
 'fc_use_laynorm_inp': True,
 'fc_use_batchnorm_inp': False,
 'fc_act': ['prelu', 'prelu', 'prelu']}

In [13]:
DNN2_arch

{'input_dim': 512,
 'fc_lay': [41],
 'fc_drop': [0.0],
 'fc_use_batchnorm': [False],
 'fc_use_laynorm': [False],
 'fc_use_laynorm_inp': False,
 'fc_use_batchnorm_inp': False,
 'fc_act': ['softmax']}

In [14]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
            
        ## Splitting the name into processable data:
        l        = name.split(".")

        Net_name = l[0]
        fun      = l[1]
        num      = int(l[2]) if len(l[2])==1 else -1
        
        

        if('bn' in fun):
            if('CNN' in Net_name):
                if(not CNN_arch['cnn_use_batchnorm'][num]):
                    continue
            elif('DNN1' in Net_name):
                if(not DNN1_arch['fc_use_batchnorm'][num]):
                    continue
            elif('DNN2' in Net_name):
                if(not DNN2_arch['fc_use_batchnorm'][num]):
                    continue
            
        if('ln' in fun):
            if('0' in fun):
                if('CNN' in Net_name):
                    if(not CNN_arch['cnn_use_laynorm_inp']):
                        continue
                elif('DNN1' in Net_name):
                    if(not DNN1_arch['fc_use_laynorm_inp']):
                        continue
                elif('DNN2' in Net_name):
                    if(not DNN2_arch['fc_use_laynorm_inp']):
                        continue
            else:
                if('CNN' in Net_name):
                    if(not CNN_arch['cnn_use_laynorm'][num]):
                        continue
                elif('DNN1' in Net_name):
                    if(not DNN1_arch['fc_use_laynorm'][num]):
                        continue                        
                elif('DNN2' in Net_name):
                    if(not DNN2_arch['fc_use_laynorm'][num]):
                        continue
            
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params
    

In [15]:
count_parameters(Main_net)# in_features=56220 * out_features=2048 = 115,138,560
# in_features_4layCNN = 18660 = 312 * 60 = (935 -3 +1) /3 * Nbre_Filt
# in_features_4layCNN = 18660 * out_features=1516 = 28,288,560  

+-------------------------+------------+
|         Modules         | Parameters |
+-------------------------+------------+
|  CNN_net.conv.0.low_hz_ |     80     |
| CNN_net.conv.0.band_hz_ |     80     |
|  CNN_net.conv.1.weight  |     90     |
|   CNN_net.conv.1.bias   |     10     |
|  CNN_net.conv.2.weight  |    1800    |
|   CNN_net.conv.2.bias   |     20     |
|  CNN_net.conv.3.weight  |    3600    |
|   CNN_net.conv.3.bias   |     20     |
|   CNN_net.ln.0.weight   |  2553280   |
|    CNN_net.ln.0.bias    |  2553280   |
|   CNN_net.ln.1.weight   |   137670   |
|    CNN_net.ln.1.bias    |   137670   |
|   CNN_net.ln.2.weight   |   42120    |
|    CNN_net.ln.2.bias    |   42120    |
|   CNN_net.ln.3.weight   |   18240    |
|    CNN_net.ln.3.bias    |   18240    |
|    CNN_net.ln0.weight   |   96000    |
|     CNN_net.ln0.bias    |   96000    |
|   DNN1_net.wx.0.weight  |  18677760  |
|    DNN1_net.wx.0.bias   |    1024    |
|   DNN1_net.wx.1.weight  |  1048576   |
|    DNN1_net.wx

26016140

In [17]:
from torchsummary import summary

#summary(CNN_net, (1, wlen))

Main_net.state_dict()

OrderedDict([('CNN_net.conv.0.low_hz_',
              tensor([[   30.0000],
                      [   59.0715],
                      [   89.3007],
                      [  120.7338],
                      [  153.4186],
                      [  187.4051],
                      [  222.7451],
                      [  259.4925],
                      [  297.7032],
                      [  337.4357],
                      [  378.7505],
                      [  421.7106],
                      [  466.3816],
                      [  512.8315],
                      [  561.1312],
                      [  611.3544],
                      [  663.5778],
                      [  717.8808],
                      [  774.3464],
                      [  833.0607],
                      [  894.1132],
                      [  957.5971],
                      [ 1023.6092],
                      [ 1092.2501],
                      [ 1163.6246],
                      [ 1237.8414],
                      [ 